In [ ]:
from pathlib import Path

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.nn.utils import clip_grad_norm_
from tqdm.auto import tqdm

from src.dataloader import DataLoader
from src.model import EnokeeConfig, EnokeeEncoder
from src.tokenizer import LUKETokenizer
from src.utils import (get_num_param_and_model_size, classification_metrics,
                       load_checkpoint, save_checkpoint)

torch.manual_seed(0)
torch.cuda.manual_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16

In [ ]:
def train(output_dir, dataloader, model, optimizer, epochs, 
          scheduler=None, logger=None, previous_state=(0, 0), 
          clip_val=5, save_every=100, log_every=10):
    
    print("INFO: Using device {}".format(str(device)))
    print("INFO: Starting training, press CTRL+C to stop")
    
    # setup model
    model.to(device, dtype)
    model.train()
    get_num_param_and_model_size(model)
    
    # setup
    step, epoch = previous_state
    criterion = torch.nn.NLLLoss(ignore_index=-1)
    # criterion = torch.nn.CrossEntropyLoss(ignore_index=-1, label_smoothing=0.15)
    softmax = torch.nn.functional.log_softmax
    tokenizer = LUKETokenizer()
    
    # train loop
    while epoch < epochs:
        pbar = tqdm(dataloader, desc=f"[EPOCH {epoch+1}/{epochs}]")
        for sentences, spans, targets in pbar:
            # reshape targets
            targets = targets.flatten().to(device)
            # zero grad
            optimizer.zero_grad()
            # forward pass
            try:
                inputs = tokenizer(sentences, spans).to(device)
            except IndexError:
                print(f"ERROR: Index error in batch {step}, skipping batch")
                pass
            outputs = model(**inputs)
            outputs = outputs.view(-1, model.config.num_entities)
            # loss and backward pass
            outputs = softmax(outputs, dim=1)
            loss = criterion(outputs, targets)
            # loss = criterion(outputs, targets)
            loss.backward()
            clip_grad_norm_(model.parameters(), clip_val)
            optimizer.step()
            # update learning rate
            if scheduler is not None:
                scheduler.step()    
            # log metrics
            if logger is not None and step % log_every - 1 == 0:
                mask = inputs['entity_attention_mask'].bool().flatten()
                y_true = torch.masked_select(targets, mask)
                y_pred = torch.masked_select(outputs.argmax(1), mask)
                acc, prec, rec, f1 = classification_metrics(y_true, y_pred)
                logger.add_scalar("Loss/Train", loss.item(), step)
                logger.add_scalar("Accuracy/Train", acc, step)
                logger.add_scalar("Precision/Train", prec, step)
                logger.add_scalar("Recall/Train", rec, step)
                logger.add_scalar("Macro-F1/Train", f1, step)
            # save checkpoints
            if step % save_every - 1 == 0:
                save_checkpoint(output_dir, step, epoch, dataloader, model, optimizer, scheduler)
            # update progress bar
            pbar.set_postfix({"loss": loss.item()})
            pbar.update(dataloader.batch_size)
            # global step
            step += 1
            
        # global epoch
        epoch += 1

In [ ]:
def main(output_dir, dataset_path, default_output_dir, batch_size, epochs, dataset_len=0):
    # initialise dataloader, model, optimizer and (optionally schedular)
    step = 0
    epoch = 0
    dataloader = None
    config = EnokeeConfig(num_entities=14065, d_ff=3072, n_layers=1, finetune=True)
    model = EnokeeEncoder(config).to(device=device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = None

    # load checkpoints if exist
    if output_dir is not None:
        print("INFO: Loading checkpoints")
        output_dir = Path(output_dir)
        (step, 
         epoch, 
         dataloader_state_dict, 
         model_state_dict, 
         optimizer_state_dict,
         scheduler_state_dict,) = load_checkpoint(output_dir, device)
        # load dataloader_state_dict
        dataloader = DataLoader.from_state_dict(dataloader_state_dict)
        dataloader.batch_size = batch_size
        # load model_state_dict
        model.load_state_dict(model_state_dict, strict=False)
        # load optimizer_state_dict
        optimizer.load_state_dict(optimizer_state_dict)
        # load scheduler_state_dict
        if scheduler is not None and scheduler_state_dict is not None:
            scheduler.load_state_dict(scheduler_state_dict)

    elif dataset_path is not None:
        print("INFO: Loading dataset")
        dataset_path = Path(dataset_path)
        output_dir = Path(default_output_dir)
        output_dir.mkdir(exist_ok=True)
        # create dataloader
        if dataset_path.exists():
            dataloader = DataLoader(dataset_path, batch_size=batch_size, nrows=dataset_len)
        else:
            raise FileNotFoundError("Dataset does not exist at the provided path")
        # remove previous tensorboard files
        print("INFO: existing checkpoints found, removing...")
        if output_dir.exists():
            for file in output_dir.iterdir():
                file.unlink()
            output_dir.rmdir()
    else:
        raise ValueError(
            "No arguments provided, run `python train.py --help to list arguments"
        )

    # initialise summary writer
    logger = SummaryWriter(output_dir)
    # train
    try:
        train(output_dir, dataloader, model, optimizer, epochs, 
              scheduler=scheduler, logger=logger, previous_state=(step, epoch),
              save_every=10, log_every=10)
    except KeyboardInterrupt:
        if logger is not None:
            logger.close()
        print("Stopped training.")

In [ ]:
main(output_dir=None,
     dataset_path="./data/zelda.jsonl.bz2",
     dataset_len=22535558,
     default_output_dir="./output",
     batch_size=2,
     epochs=5)